# Laboratorio 10
## Visite in ampiezza e profondità - BFS & DFS

### BFS
*BFS (Breadth-First Search)* è un algoritmo di ricerca in ampiezza che esplora gradualmente il grafo, visitando tutti i nodi a una data profondità prima di passare alla profondità successiva. In altre parole, l'algoritmo esplora il grafo in modo **orizzontale**, visitando tutti i nodi adiacenti al nodo di partenza prima di passare ai nodi adiacenti ai nodi visitati. Si utilizza una **coda** per mantenere traccia dei nodi visitati e visitare ogni nodo solo una volta. 

**Domande:**
1. In quale forma solitamente viene rappresentato un grafo di input per BFS?
2. Qual è il tempo di esecuzione di BFS se rappresentiamo il suo grafo di input con una matrice di adiacenza e modifichiamo l'algoritmo per gestire questa forma di input?

**Esercizio - Diametro di un albero d-ario utilizzando BFS**

Dato un albero d-ario, è necessario calcolare la lunghezza del diametro dell'albero.
Il diametro di un albero d-ario è la lunghezza del percorso più lungo tra due nodi qualsiasi dell'albero.

*Hint:* L'albero può essere rappresentato come una lista di adiacenza.

In [9]:
from collections import deque

class TreeNode:
    def __init__(self, val=0, children=None):
        self.val = val
        self.children = children if children else []

def tree_diameter(root: TreeNode) -> int:
    """
    La funzione riceve in input il nodo radice di un albero d-ario e restituisce la lunghezza del diametro dell'albero.
    """
    if not root:
        return 0
    
    # Eseguiamo la prima BFS per trovare il nodo più lontano dalla radice
    node1, _ = bfs(root)
    
    # Eseguiamo la seconda BFS partendo dal nodo più lontano trovato in precedenza
    node2, diameter = bfs(node1)
    
    # Restituiamo l'ultima distanza trovata, che corrisponderà al diamtero dell'albero
    return diameter

def bfs(root: TreeNode) -> tuple[TreeNode, int]:
    """
    La funzione riceve in input il nodo radice di un albero d-ario e restituisce il nodo più lontano dalla radice e la sua distanza.
    """
    queue = deque([(root, 0)])
    visited = set([root])
    node, dist = root, 0
    
    while queue:
        curr_node, curr_dist = queue.popleft()
        
        # Se la distanza corrente è maggiore della distanza massima trovata finora,
        # aggiorniamo il nodo e la distanza massima
        if curr_dist > dist:
            node, dist = curr_node, curr_dist
            
        # Aggiungiamo i figli del nodo corrente alla coda
        for child in curr_node.children:
            if child not in visited:
                queue.append((child, curr_dist + 1))
                visited.add(child)
                
    return node, dist

In [14]:
root = TreeNode(1)
node2 = TreeNode(2)
node3 = TreeNode(3)
node4 = TreeNode(4)
node5 = TreeNode(5)
node6 = TreeNode(6)
node7 = TreeNode(7)

root.children.extend([node2, node3, node6])
node2.children.extend([root, node4, node5])
node3.children.append(root)
node6.children.extend([root, node7])
node4.children.append(node2)
node5.children.append(node2)
node7.children.append(node6)

#        1
#     /  |  \
#    2   3   6
#   / \       \
#  4   5       7

diameter = tree_diameter(root)
diameter

4

*Follow-up*: Settando d=2, quindi nel caso di un albero binario, quale sarebbe la complessità di tale algoritmo?

### Grafi bipartiti

Un grafo è bipartito se i suoi nodi possono essere divisi in due insiemi disgiunti $A$ e $B$ tali che ogni arco del grafo collega un nodo di $A$ ad un nodo di $B$ e nessun arco collega due nodi dello stesso insieme.

La definizione è equivalente a dire che il grafo è bicromatico, ovvero che è possibile colorare i nodi con due colori in modo che ogni arco colleghi due nodi di colori diversi.

**OSS**: Un grafo è bipartito se e solo se non contiene cicli di lunghezza dispari.

* Mostrare che un grafo è bipartito se e solo se non contiene cicli di lunghezza dispari.
* Scrivere una funzione `grafo_bipartito` che prende in input un grafo e restituisce `True` se il grafo è bipartito e `False` altrimenti.
* Scrivere un algoritmo che restituisce il numero di cicli di un grafo.

Soluzione:

1. 
    $\rightarrow$: ovvio, se prendiamo un ciclo di lunghezza dispari, allora il ciclo deve contenere un numero dispari di nodi, ma la bipartizione implica che il numero di nodi sia pari.

    $\leftarrow$: se il grafo non contiene cicli di lunghezza dispari, allora è bipartito. 
    Tutti i cicli di lunghezza pari possono essere colorati con due colori alternati, quindi il grafo può essere bi-colorato.
    Per mostrarlo, possiamo partire da un nodo arbitrario e colorarlo, poi procedere con una visita in ampiezza colorando i nodi adiacenti con il colore opposto. Se durante la visita si incontra un nodo già colorato con lo stesso colore del nodo corrente, allora il grafo non è bipartito, altrimenti lo è. 

2. 
    ```python
    def grafo_bipartito(G, s):
        '''
        G: grafo 
        s: nodo sorgente
        '''
        Q <- Queue()
        Q.enqueue(s)
        color <- array(len(G))
        for v in G:
            color[v] <- None
        color[s] <- 0
        
        while not Q.is_empty():
            u <- Q.dequeue()
            for v in G.adj[u]:
                if color[v] is None:
                    color[v] <- 1 - color[u]
                    Q.enqueue(v)
                elif color[v] == color[u]:
                    return False
        return True
    ```

3. Scrivere un algoritmo che restituisce il numero di cicli in un grafo. Per farlo, possiamo usare una variante della DFS che tiene traccia dei nodi già visitati e dei nodi che sono stati visitati ma non sono ancora stati completati. Se durante la visita si incontra un nodo già visitato ma non completato, allora si è trovato un ciclo.

    ```python

    
    def numero_cicli(G):
        '''
        G: grafo 
        '''
        visited <- array(len(G))
        completed <- array(len(G))
        count <- 0
        for v in G:
            visited[v] <- False
            completed[v] <- False
        for v in G:
            if not visited[v]:
                count <- count + dfs(G, v, visited, completed)
        return count

    def dfs(G, u, visited, completed):
        '''
        G: grafo
        u: nodo corrente
        visited: array di booleani
        completed: array di booleani
        '''
        visited[u] <- True
        for v in G.adj[u]:
            if not visited[v]:
                return dfs(G, v, visited, completed)
            elif not completed[v]:
                return 1
        completed[u] <- True
        return 0
    ```
Modificando questo algoritmo tenendo conto del tempo di inizio e fine visita di ogni nodo, si può stabilire la lunghezza del ciclo e pertanto usare questo algoritmo per rispondere alla domanda se il grafo sia bipartito o meno.